In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
train_df = pd.read_csv('train_ver2.csv')
test_df = pd.read_csv('test_ver2.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_14992\3574963415.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_ver2.csv')
C:\Users\USER\AppData\Local\Temp\ipykernel_14992\3574963415.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test_ver2.csv')


In [4]:
print(train_df.shape)
train_df.head()

(13647309, 48)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

In [6]:
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"}

train_df.rename(col_names, axis = 1, inplace = True)

In [7]:
numeric_cols = ['age', 'income', 'seniority']
date_cols = ['first_date', 'fecha_dato']

train_df[numeric_cols] = train_df[numeric_cols].apply(pd.to_numeric, errors='coerce')
train_df[date_cols] = train_df[date_cols].apply(pd.to_datetime, errors='coerce')


In [8]:
train_df.isnull().mean()*100

fecha_dato            0.000000
cust_id               0.000000
emp_index             0.203220
residence             0.203220
sex                   0.203732
age                   0.203220
first_date            0.203220
new_cust              0.203220
seniority             0.203220
is_primary            0.203220
last_primary_date    99.818330
cust_type             1.097513
cust_rel_type         1.097513
residence_index       0.203220
foreigner_index       0.203220
spouse_index         99.986752
channel               1.363829
indfall               0.203220
tipodom               0.203227
province              0.685784
province_name         0.685784
active_index          0.203220
income               20.475648
segment               1.387585
ind_ahor_fin_ult1     0.000000
ind_aval_fin_ult1     0.000000
ind_cco_fin_ult1      0.000000
ind_cder_fin_ult1     0.000000
ind_cno_fin_ult1      0.000000
ind_ctju_fin_ult1     0.000000
ind_ctma_fin_ult1     0.000000
ind_ctop_fin_ult1     0.000000
ind_ctpp

In [9]:
train_df.drop(columns=['last_primary_date','spouse_index'], inplace=True)
test_df.drop(columns=['last_primary_date','spouse_index'], inplace=True)

In [10]:
cols = ['emp_index','residence','sex','first_date','new_cust','is_primary',"cust_type","cust_rel_type",
       "province","province_name","active_index","channel","segment"]

for i in cols:
    train_df.loc[train_df[i].isnull(), i] = train_df[i].value_counts().index[0]
    test_df.loc[test_df[i].isnull(), i] = test_df[i].value_counts().index[0]

train_df['income'].fillna(train_df['income'].mean(), inplace = True)
test_df['income'].fillna(test_df['income'].mean(), inplace = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_14992\2432065694.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['income'].fillna(train_df['income'].mean(), inplace = True)
C:\Users\USER\AppData\Local\Temp\ipykernel_14992\2432065694.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

# popularity based 

考量到冷啟動階段，剛開始先使用基於流行度的模型進行推薦。同時要注意有些產品其實是在更早之前就進行熱銷的，這或許會影相新一年度的產品推薦決策，因此可以使用time decay function(時間衰減函數)來進行調整。方法主要是為近期的互動賦予更多權重，並減少就互動的影響。

In [11]:
def popularity(df):
    popular_col = {}
    for i in df.columns[22:]:
        count_one = df[i].value_counts()[1]
        ratio = np.round(count_one/df.shape[0], decimals=2)
        popular_col[i] = ratio
        
        popular_col_sor = dict(sorted(popular_col.items(), key=lambda x: x[1], reverse=True))
    
    return popular_col_sor

In [12]:
popularity(train_df)

{'ind_cco_fin_ult1': 0.66,
 'ind_ctop_fin_ult1': 0.13,
 'ind_recibo_ult1': 0.13,
 'ind_cno_fin_ult1': 0.08,
 'ind_ecue_fin_ult1': 0.08,
 'ind_nom_pens_ult1': 0.06,
 'ind_reca_fin_ult1': 0.05,
 'ind_nomina_ult1': 0.05,
 'ind_ctpp_fin_ult1': 0.04,
 'ind_dela_fin_ult1': 0.04,
 'ind_tjcr_fin_ult1': 0.04,
 'ind_valo_fin_ult1': 0.03,
 'ind_fond_fin_ult1': 0.02,
 'ind_ctju_fin_ult1': 0.01,
 'ind_ctma_fin_ult1': 0.01,
 'ind_hip_fin_ult1': 0.01,
 'ind_plan_fin_ult1': 0.01,
 'ind_ahor_fin_ult1': 0.0,
 'ind_aval_fin_ult1': 0.0,
 'ind_cder_fin_ult1': 0.0,
 'ind_deco_fin_ult1': 0.0,
 'ind_deme_fin_ult1': 0.0,
 'ind_pres_fin_ult1': 0.0,
 'ind_viv_fin_ult1': 0.0}

# memory based - item based collaborative filtering

老客戶並不適合基於流行的推薦系統，他們更適合個人化的推薦系統。因此使用協同過濾技術來創建推薦模型。在此使用item based的協同過濾方法。此方法會根據產品之間的相似性進行推薦，並根據使用者對專案的偏好來確定推薦。

In [37]:
df_ui = train_df[train_df['fecha_dato'].dt.year == 2016]
df_ui = df_ui.iloc[:,[1] + list(range(22, 46))]
df_ui = df_ui.groupby("cust_id").sum()
df_ui = df_ui.fillna(0)
df_ui.index = df_ui.index.astype('string')

df_item = df_ui.copy()
df_item = df_item[:10000].T

df_item = pd.DataFrame(cosine_similarity(df_item), index = df_item.index.astype('string'), columns = df_item.index.astype('string'))
df_item.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ind_ahor_fin_ult1,1.000000,0.000000,0.021471,0.000000,0.000000,0.0,0.000000,0.010602,0.000000,0.000000,...,0.000000,0.021264,0.000000,0.013520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ind_aval_fin_ult1,0.000000,1.000000,0.014314,0.000000,0.026139,0.0,0.000000,0.000000,0.035457,0.000000,...,0.000000,0.000000,0.000000,0.027040,0.000000,0.029972,0.000000,0.032115,0.029625,0.021234
ind_cco_fin_ult1,0.021471,0.014314,1.000000,0.038210,0.101260,0.0,0.041104,0.464800,0.230290,0.032444,...,0.083631,0.172641,0.042294,0.264723,0.244886,0.320389,0.095347,0.125622,0.135337,0.372657
ind_cder_fin_ult1,0.000000,0.000000,0.038210,1.000000,0.049255,0.0,0.010924,0.037736,0.029694,0.000000,...,0.019900,0.048973,0.000000,0.035384,0.049251,0.069028,0.000000,0.054127,0.049930,0.054682
ind_cno_fin_ult1,0.000000,0.026139,0.101260,0.049255,1.000000,0.0,0.017238,0.179937,0.347925,0.026139,...,0.217758,0.246348,0.056745,0.507433,0.571442,0.294894,0.075580,0.766189,0.839852,0.713549


In [38]:
def item_recommender(cust_id):
    
    cust_id = str(cust_id)
    
    df_cf_user_2 = df_ui.T
    top_product = df_cf_user_2[cust_id]
    top_product = top_product[top_product > 0].sort_values(ascending = False).index[0]

    if len(top_product) == 0:
        return 0

    item_sim = df_item[top_product]

    #recommend_list = item_sim[(item_sim < 1)].sort_values(ascending = False)
    recommend_list = item_sim[(item_sim < 1)]
    recommend_list = recommend_list.to_dict()
    
    return recommend_list

In [39]:
item_recommender(cust_id = "15907")

{'ind_ahor_fin_ult1': 0.02147058973765762,
 'ind_aval_fin_ult1': 0.014313726491771747,
 'ind_cco_fin_ult1': 0.9999999999999993,
 'ind_cder_fin_ult1': 0.0382099490227538,
 'ind_cno_fin_ult1': 0.10126035216991856,
 'ind_ctju_fin_ult1': 0.0,
 'ind_ctma_fin_ult1': 0.0411041059581589,
 'ind_ctop_fin_ult1': 0.46480003790874164,
 'ind_ctpp_fin_ult1': 0.23029006924800094,
 'ind_deco_fin_ult1': 0.03244444671468263,
 'ind_deme_fin_ult1': 0.04480905162571769,
 'ind_dela_fin_ult1': 0.2493935650530834,
 'ind_ecue_fin_ult1': 0.3187266203153043,
 'ind_fond_fin_ult1': 0.2632206562920751,
 'ind_hip_fin_ult1': 0.08363098892663925,
 'ind_plan_fin_ult1': 0.1726408242971609,
 'ind_pres_fin_ult1': 0.04229435902043405,
 'ind_reca_fin_ult1': 0.2647226916371082,
 'ind_tjcr_fin_ult1': 0.24488641587595494,
 'ind_valo_fin_ult1': 0.3203894652882251,
 'ind_viv_fin_ult1': 0.09534747496824543,
 'ind_nomina_ult1': 0.1256215824837543,
 'ind_nom_pens_ult1': 0.13533665561588018,
 'ind_recibo_ult1': 0.37265661620262147}

item based模型是基於客戶之前購買的產品所做出的推薦，相比於user based，可以更快反應目標客戶偏好快速變化。


# model based 

基於模型的推薦系統不僅僅依賴項目之間的相似性，而是使用機器學習進行預測。它能解決資料稀疏的問題並且可以更高效的進行預測。

In [17]:
import surprise

from surprise import Dataset, Reader

from surprise.prediction_algorithms.matrix_factorization import SVD

from surprise.model_selection import cross_validate

from surprise import accuracy

In [40]:
df_model_ui = df_ui.copy()

new_columns = {}
for i, col in enumerate(df_model_ui.columns):
    new_columns[col] = i

df_model_ui = df_model_ui.rename(columns=new_columns)

In [41]:
uim_arr = np.array(df_model_ui)

for row,item in enumerate(uim_arr):
    for column,item_value in enumerate(item):
        uim_arr[row, column] = uim_arr[row, column] / sum(item)
        
user_item_ratio_matrix = pd.DataFrame(uim_arr, columns=df_model_ui.columns, index=df_model_ui.index)

user_item_ratio_matrix = user_item_ratio_matrix.fillna(0)

user_item_ratio_matrix

C:\Users\USER\AppData\Local\Temp\ipykernel_14992\4183475389.py:5: RuntimeWarning: invalid value encountered in scalar divide
  uim_arr[row, column] = uim_arr[row, column] / sum(item)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
cust_id,,,,,,,,,,,,,,,,,,,,,
15889,0.0,0.0,0.277778,0.0,0.000,0.0,0.0,0.0,0.376569,0.0,...,0.0,0.000000,0.0,0.000000,0.346647,0.833195,0.0,0.00000,0.000000,0.000000
15890,0.0,0.0,0.000000,0.0,0.125,0.0,0.0,0.0,0.142349,0.0,...,0.0,0.196599,0.0,0.000000,0.242376,0.000000,0.0,0.31503,0.446965,0.753748
15892,0.0,0.0,0.142857,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.243828,0.317458,0.451770,0.0,0.00000,0.000000,0.766948
15893,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.00000,0.000000,0.000000
15894,0.0,0.0,0.125000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.165195,0.196599,0.242376,0.0,0.31503,0.446965,0.753748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553685,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
1553686,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
1553687,0.0,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000


In [42]:
user_item_ratio_stacked = user_item_ratio_matrix.stack().to_frame()
user_item_ratio_stacked['cust_id'] = [index[0] for index in user_item_ratio_stacked.index]
user_item_ratio_stacked['service_opted'] = [index[1] for index in user_item_ratio_stacked.index]
user_item_ratio_stacked.reset_index(drop=True, inplace=True)

user_item_ratio_stacked.rename(columns={0:"service_selection_ratio"}, inplace=True)
user_item_ratio_stacked = user_item_ratio_stacked[['cust_id','service_opted', 'service_selection_ratio']]
user_item_ratio_stacked.drop(user_item_ratio_stacked[user_item_ratio_stacked['service_selection_ratio']==0].index, inplace=True)
user_item_ratio_stacked.reset_index(drop=True, inplace=True)

user_item_ratio_stacked

,cust_id,service_opted,service_selection_ratio
0,15889,2,0.277778
1,15889,8,0.376569
2,15889,18,0.346647
3,15889,19,0.833195
4,15890,4,0.125000
...,...,...,...
1325329,1553541,2,1.000000
1325330,1553552,6,1.000000
1325331,1553559,2,1.000000
1325332,1553565,2,1.000000


In [43]:
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,1), skip_lines=1)

data = Dataset.load_from_df(user_item_ratio_stacked, reader=reader)

In [44]:
svd = SVD()

svd_results = cross_validate(algo=svd, data=data, cv=4, verbose=True)

svd_results

Evaluating RMSE, MAE of algorithm SVD on 4 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Mean    Std     
RMSE (testset)    0.3322  0.3325  0.3352  0.3361  0.3340  0.0017  
MAE (testset)     0.2336  0.2352  0.2338  0.2351  0.2344  0.0007  
Fit time          9.66    7.91    8.19    8.02    8.44    0.71    
Test time         2.57    1.10    1.05    1.61    1.58    0.61    


{'test_rmse': array([0.33221633, 0.33249634, 0.33519521, 0.33613532]),
 'test_mae': array([0.23359873, 0.23519984, 0.23379366, 0.23513225]),
 'fit_time': (9.655391216278076,
  7.91381573677063,
  8.194132804870605,
  8.015746831893921),
 'test_time': (2.5748798847198486,
  1.1019618511199951,
  1.0519208908081055,
  1.6068177223205566)}

In [45]:
trainset = data.build_full_trainset()

svd = SVD()
svd.fit(trainset)

In [46]:
pd.DataFrame(list(trainset.all_ratings()))

,0,1,2
0,0,0,0.277778
1,0,1,0.376569
2,0,2,0.346647
3,0,3,0.833195
4,1,4,0.125000
...,...,...,...
1325329,705916,0,1.000000
1325330,705917,19,1.000000
1325331,705918,0,1.000000
1325332,705919,0,1.000000


In [49]:
def model_recommendation(cust_id):
    recommendations = [(sid, svd.predict(cust_id, sid).est) for sid in range(24)]
    # Convert to dictionary
    recommendations_dict = {rec[0]: rec[1] for rec in recommendations}

    labels_mapping = {idx: col for idx, col in enumerate(df_item.columns)}
    recommendations_dict = {labels_mapping[key]: value for key, value in recommendations_dict.items()}

    return recommendations_dict

In [53]:
model_recommendation(15890)

{'ind_ahor_fin_ult1': 0.4672696926471614,
 'ind_aval_fin_ult1': 0.3197183031894161,
 'ind_cco_fin_ult1': 0.9784125708419394,
 'ind_cder_fin_ult1': 0.5277285126597303,
 'ind_cno_fin_ult1': 0.407009569913396,
 'ind_ctju_fin_ult1': 0.9835502755879311,
 'ind_ctma_fin_ult1': 0.815251733814941,
 'ind_ctop_fin_ult1': 0.6949689330007638,
 'ind_ctpp_fin_ult1': 0.6435784698337668,
 'ind_deco_fin_ult1': 0.452907344972844,
 'ind_deme_fin_ult1': 0.5768893009230943,
 'ind_dela_fin_ult1': 0.817827188566681,
 'ind_ecue_fin_ult1': 0.5511938227288088,
 'ind_fond_fin_ult1': 0.730055759757134,
 'ind_hip_fin_ult1': 0.40732912735037224,
 'ind_plan_fin_ult1': 0.5681773482248593,
 'ind_pres_fin_ult1': 0.7738591441146236,
 'ind_reca_fin_ult1': 0.5748322701467453,
 'ind_tjcr_fin_ult1': 0.49138075904030787,
 'ind_valo_fin_ult1': 0.6987174789285001,
 'ind_viv_fin_ult1': 0.7990398188837586,
 'ind_nomina_ult1': 0.43968582389719296,
 'ind_nom_pens_ult1': 0.5773012182206304,
 'ind_recibo_ult1': 0.6595676503693175}

# weight hybrid recommendation

In [55]:
def hybrid_weight(cust_id, df_p,f1, f2, f3):
    pb_w = popularity(df_p)
    it_w = item_recommender(cust_id)
    md_w = model_recommendation(cust_id)

    hybrid = {k: (v * f1) + (it_w[k] * f2) + (md_w[k] * f3) for k, v in pb_w.items()}

    return hybrid

    

In [57]:
hybrid_outcome_15890 = hybrid_weight(15890, df_p= train_df, f1 = 0.25, f2 = 0.25, f3= 0.5)

In [74]:
def recommendation_system(cust_id, df, hybrid_outcome, top_n):

    user_products = [product for product in df.columns if df.loc[cust_id, product] == 1]
                
    recom = { key : hybrid_outcome[key] for key in hybrid_outcome if key not in user_products}

    recom_sort = dict(sorted(recom.items(), key=lambda item: item[1], reverse=True))

    top_n_dict = {key: recom_sort[key] for key in list(recom_sort)[:3]}

    top_n_list = list(top_n_dict.keys())

    top_n_df = pd.DataFrame(top_n_list, columns=['Recommendation product'])

    return top_n_df

In [75]:
recommendation_system(15890, train_df,hybrid_outcome_15890, 3 )

,Recommendation product
0,ind_cco_fin_ult1
1,ind_nom_pens_ult1
2,ind_ctju_fin_ult1
